In [ ]:
import pandas as pd
import numpy as np
import json
from itertools import chain
import os

def GetResultsFromJSON(ticker, fold, exp_dir):
    result_dir = f"{exp_dir}/fold{fold}/{ticker}/model_losses.json"
    with open(result_dir, 'r') as f:
        results = json.load(f)
    return results
def FetchBestPreds(results):
    best_idx = np.argmin(results['val_losses'])
    return results['val_means'][best_idx], results['val_stds'][best_idx]
# from directory, get each ticker and make a list of tickers
ticker_dir = 'exp_base/fold1/'
tickers = []
for ticker in os.listdir(ticker_dir):
    tickers.append(ticker)


# Empty list to store each ticker's results as a series
series_list = []

for ticker in tickers:
    results = GetResultsFromJSON(ticker, 1, 'exp_base')
    val_means, val_stds = FetchBestPreds(results)
    # Flatten the list of lists
    #val_means = list(chain.from_iterable(results['val_means']))
    # add business dates to results
    results['dates'] = pd.bdate_range(start='2016-01-01', periods=len(val_means))
    # Convert dates to pandas datetime and add as index to series
    s = pd.Series(val_means, index=pd.to_datetime(results['dates']), name=ticker)
    # add val_stds to series
    s['val_stds'] = val_stds
    series_list.append(s)

# Concatenate all series along the columns
df_results = pd.concat(series_list, axis=1)
# remove all rows with NaNs
df_results = df_results.dropna(axis=0, how='any')
print(df_results)

In [ ]:
print(df_results.index)
print(df_results_stds.index)

In [ ]:
import cvxpy as cp


n = len(tickers)

# Here gamma is a risk tolerance factor which you can tune
gamma = 0.1

# Initialize an empty dataframe to hold the portfolio weights
weights_df = pd.DataFrame(columns=df_results.columns)
# Initialize an empty list to store the average returns
avg_returns = []
avg_stds = []
for i in range(len(df_results)):
    # Take the predictions for the day
    mu = df_results.iloc[i].values

    # Calculate the covariance matrix
    Sigma = df_results_stds.iloc[i].values

    # Convert Sigma to covariance matrix
    Sigma = np.diag(Sigma)

    # Initialize the weights variable
    w = cp.Variable(n)


    # Formulate the portfolio optimization problem
    prob = cp.Problem(cp.Minimize(-mu.T@w + gamma*cp.quad_form(w, Sigma)),
                    [cp.sum(w) == 1,
                    w >= 0, 
                    w <= 0.4])

    # # Formulate the portfolio optimization problem
    # prob = cp.Problem(cp.Minimize(-mu.T@w + gamma*cp.quad_form(w, Sigma)),
    #                   [cp.sum(w) == 1,
    #                   w >= 0, w <= 0.4])

    # Solve the problem
    prob.solve()

    # Add the optimal weights to the dataframe
    weights_df = weights_df.append(pd.Series(w.value, index=df_results.columns, name=df_results.index[i]))

    # Calculate the weighted average return using the weights and the mean returns for the day
    avg_return = np.sum(weights_df.iloc[i] * df_results.iloc[i])
    # Append the average return to the list
    avg_returns.append(avg_return)

    # calculate the weighted average standard deviation using the weights and the stds for the day
    avg_std = np.sum(weights_df.iloc[i] * df_results_stds.iloc[i])

    # append the average std to the list
    avg_stds.append(avg_std)

    



# Add the average returns as a new column to weights_df
weights_df['Predicted Average Return (%)'] = avg_returns
# multiply average return by 100 to get percentages
weights_df['Predicted Average Return (%)'] = weights_df['Predicted Average Return (%)'] * 100
# Add the average stds as a new column to weights_df
weights_df['Predicted Average Std'] = avg_stds

# rename the columns of the tickers to the ticker name and then _weight
weights_df.columns = [f"{col}_weight" for col in weights_df.columns[:-2]] + ['Predicted Average Return (%)', 'Predicted Average Std']

round(weights_df,2)


 

Portfolio optimization with risk constraint included:

In [ ]:
import cvxpy as cp


n = len(tickers)

# Here gamma is a risk tolerance factor which you can tune
gamma = 0.1

# Initialize an empty dataframe to hold the portfolio weights
weights_df = pd.DataFrame(columns=df_results.columns)
# Initialize an empty list to store the average returns
avg_returns = []
avg_stds = []
for i in range(len(df_results)):
    # Take the predictions for the day
    mu = df_results.iloc[i].values

    # Calculate the covariance matrix
    Sigma = df_results_stds.iloc[i].values

    # Convert Sigma to covariance matrix
    Sigma = np.diag(Sigma)

    # Initialize the weights variable
    w = cp.Variable(n)

    # Risk constraint
    portfolio_variance = cp.quad_form(w, Sigma)
    risk_limit = 0.05  # Set your risk limit here
    risk_constraint = [portfolio_variance <= risk_limit]

    # Formulate the portfolio optimization problem
    prob = cp.Problem(cp.Maximize(mu.T @ w - gamma * portfolio_variance),
                      [cp.sum(w) == 1,
                       w >= 0, 
                       w <= 0.4] + risk_constraint)

    # Solve the problem
    prob.solve()

    # Add the optimal weights to the dataframe
    weights_df = weights_df.append(pd.Series(w.value, index=df_results.columns, name=df_results.index[i]))

    # Calculate the weighted average return using the weights and the mean returns for the day
    avg_return = np.sum(weights_df.iloc[i] * df_results.iloc[i])
    # Append the average return to the list
    avg_returns.append(avg_return)

    # calculate the weighted average standard deviation using the weights and the stds for the day
    avg_std = np.sum(weights_df.iloc[i] * df_results_stds.iloc[i])

    # append the average std to the list
    avg_stds.append(avg_std)

# Add the average returns as a new column to weights_df
weights_df['Predicted Average Return (%)'] = avg_returns
# multiply average return by 100 to get percentages
weights_df['Predicted Average Return (%)'] = weights_df['Predicted Average Return (%)'] * 100
# Add the average stds as a new column to weights_df
weights_df['Predicted Average Std'] = avg_stds

# rename the columns of the tickers to the ticker name and then _weight
weights_df.columns = [f"{col}_weight" for col in weights_df.columns[:-2]] + ['Predicted Average Return (%)', 'Predicted Average Std']

round(weights_df,2)


In [ ]:
# get the sp500 data loaded
stock_data = pd.read_csv('SP500_stock_prices.csv', index_col=0, parse_dates=True)
# Using the closing price, for each stock calculate the return 63 business days in the future
# and store it in a new column called 'Future Return'
stock_data['63-Day Return (%)'] = stock_data.groupby('Ticker')['Close'].transform(lambda x: x.pct_change(63).shift(-63)) * 100
# Drop all rows with NaNs
stock_data = stock_data.dropna(axis=0, how='any')

In [ ]:
stock_data

In [ ]:
pivoted_stock_data = stock_data.pivot_table(index='Date', columns='Ticker', values='63-Day Return (%)')
# Join the weights_df with the pivoted_stock_data on the date index
temporary_df = weights_df.join(pivoted_stock_data)

# Initialize a new column for the actual portfolio return
temporary_df['Actual Portfolio Return (%)'] = 0

# For each ticker, add the weighted return to the actual portfolio return
for ticker in tickers:
    # if the ticker is not in the temporary_df, skip it
    if ticker not in temporary_df.columns:
        continue
    temporary_df['Actual Portfolio Return (%)'] += temporary_df[ticker]*temporary_df[ticker + '_weight']

# Now assign the 'Actual Portfolio Return' column back to the weights_df
weights_df['Actual Portfolio Return (%)'] = temporary_df['Actual Portfolio Return (%)']

# remove all rows with NaNs
weights_df = weights_df.dropna(axis=0, how='any')

# round the weights to 2 decimal places
weights_df = round(weights_df, 2)



In [ ]:
# remove columns that have a weight of 0
weights_df = weights_df.loc[:, (weights_df != 0).any(axis=0)]

In [ ]:
weights_df

In [ ]:
import matplotlib.pyplot as plt
# Calculate the average return for the portfolio
portfolio_return = weights_df['Actual Portfolio Return (%)'].mean()
# Calculate the standard deviation for the portfolio
portfolio_std = weights_df['Actual Portfolio Return (%)'].std()
# Calculate the Sharpe ratio for the portfolio


# Print the results
print("The average return for the portfolio is:", round(portfolio_return,2),"%")
print("The standard deviation of the portfolio return is:", round(portfolio_std,2))



In [ ]:
# caluclate the average predicted return for the portfolio
portfolio_predicted_return = weights_df['Predicted Average Return (%)'].mean()
# Calculate the standard deviation for the portfolio
portfolio_std = weights_df['Predicted Average Return (%)'].std()
# Calculate the Sharpe ratio for the portfolio
sharpe_ratio = (portfolio_predicted_return-risk_free_rate)/portfolio_std

# Print the results
print("The average predicted return for the portfolio is:", round(portfolio_predicted_return,2),"%")
print("The standard deviation of the portfolio return is:", round(portfolio_std,2))
print("The Sharpe ratio for the portfolio is:", round(sharpe_ratio,2))


In [ ]:
# plot the predicted average returns as histogram
weights_df['Predicted Average Return (%)'].plot(kind='hist', bins=20, title='Average Predicted Return (%)')
plt.show()

# plot the actual portfolio returns as histogram
weights_df['Actual Portfolio Return (%)'].plot(kind='hist', bins=20, title='Actual Portfolio Return (%)')  
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import cvxpy as cp
import matplotlib.pyplot as plt

# Your dataframes: df_returns for daily returns, df_stds for daily standard deviations
# They should have the same shape and share indices and columns
assert df_results.shape == df_results_stds.shape
assert (df_results.index == df_results_stds.index).all()
assert (df_results.columns == df_results_stds.columns).all()

# Set up an array to hold results
n_days, n_assets = df_results.shape

n_portfolios = 5000  # number of portfolios to simulate each day

# Run simulation for each day
for i in range(n_days):
    # Get mean returns and standard deviations for the day
    means = df_results.iloc[i].values
    stds = df_results_stds.iloc[i].values

    # Calculate the covariance matrix based on the predicted stds
    cov_matrix = np.diag(stds**2)

    # Arrays to store simulation results
    portfolio_returns = []
    portfolio_risks = []
    portfolio_weights = []
    portfolio_sharpe_ratios = []

    # Generate random portfolios
    for _ in range(n_portfolios):
        weights = np.random.dirichlet(np.ones(n_assets))  # generate random weights
        portfolio_weights.append(weights)

        # Calculate portfolio return and risk
        returns = np.dot(weights, means)
        portfolio_returns.append(returns)

        var = np.dot(weights.T, np.dot(cov_matrix, weights))
        portfolio_risks.append(np.sqrt(var))

        # Calculate Sharpe ratio
        sharpe_ratio = (returns - 3 / 4) / np.sqrt(var) # risk free rate in last quarter of 2019 was 3%
        portfolio_sharpe_ratios.append(sharpe_ratio)

    # Convert to numpy arrays
    portfolio_returns = np.array(portfolio_returns)
    portfolio_risks = np.array(portfolio_risks)
    portfolio_weights = np.array(portfolio_weights)
    portfolio_sharpe_ratios = np.array(portfolio_sharpe_ratios)

    # Identify the portfolio with the highest Sharpe ratio
    max_index = np.argmax(portfolio_sharpe_ratios)
    optimal_risk = portfolio_risks[max_index]
    optimal_return = portfolio_returns[max_index]
    optimal_weights = portfolio_weights[max_index]

    # Plot the efficient frontier for the day
    plt.figure(figsize=(10, 6))
    plt.scatter(portfolio_risks, portfolio_returns, c=portfolio_sharpe_ratios, cmap='viridis', marker='o')
    plt.colorbar(label='Sharpe ratio')
    # plot the minimum volatility portfolio
    plt.scatter(portfolio_risks[np.argmin(portfolio_risks)], portfolio_returns[np.argmin(portfolio_risks)], color='g', marker='*', s=500) # min volatility portfolio
    plt.scatter(optimal_risk, optimal_return, color='r', marker='*', s=500)  # optimal portfolio
    # make efficient frontier line connecting min vol portfolio and optimal portfolio

    plt.xlabel('Volatility')
    plt.ylabel('Return')
    plt.title(f'Efficient Frontier for Day {i+1}')
    plt.show()

